## Библиотека [SPACY](https://huggingface.co/spacy/ru_core_news_lg)

### [Обучение модели в части NER](https://mizakona.ru/obucenie-novyx-sushhnostei-v-modeli-spacy-ner/)

model | version
---|---
ru_core_news_md| spaCy 3.7.0

Чтобы обновить модель, необходимо будет передать ей множесто примеров, которые содержат текст, указание на сущности и их класс. В примерах необходимо использовать целые предложения, поскольку при извлечении сущностей модель во многом ориентируется на контекст предложения. </br>
Очень важно всесторонне обучить модель, чтобы она умела распознавать токены, не являющиеся сущностями.

```python
("What to expect at Apple's 10 November event",
{"entities": [(18, 23, "COMPANY")]})

("Is that apple pie I smell?",
{"entities": []})
```
Для обучения мы выделяем позиции, где начинается и заканчивается её наименование, а затем проставляем нашу метку о том, что эта сущность является компанией. Во втором примере речь идёт о фрукте, поэтому сущности отсутствуют.

```python
training_data = [
    ('What to expect at Apple's 10 November event',
    {'entities': [(18, 23, 'COMPANY')]}),
    ( ... )
]

nlp =

for i in range(10):
    random.shuffle(training_data)
    for batch in spacy.util.minibatch(training_data):
        texts = [text for text, annotation in batch]
        annotations = [annotation for text, annotation in batch]
        nlp.update(texts, annotations)

nlp.to_disk('model')
```
Чтобы модель эффективно обучалась, нужно провести серию из нескольких обучений. С каждым обучением модель будет оптимизировать веса тех или иных параметров. Модели в spaCy используют используют методику стохастического градиентного спуска, поэтому неплохим решением будет перемешивать примеры при каждом обучении, а также передавать их небольшими порциями.

```python
from spacy.training.example import Example

nlp = spacy.load('ru_core_news_md')

new_entity = '1 ruble'
label = 'MONEY'

training_data = [
    ('A lot (1 ruble) of money in SBERBANK',
    {'entities': [(10, 24, 'MONEY')]})
]

#ДОБАВЛЕНИЕ НОВОЙ СУЩНОСТИ В МОДЕЛЬ
ner = nlp.get_pipe('ner')
ner.add_label(label)

train_examples = []
for text, annotations in training_data:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotations)
    train_examples.append(example)

for epoch in range(10):
    spacy.util.sfuffle(train_examples)
    for example in train_examples:
        nlp.update([example], drop = 0.5, losses = {})

nlp.to_disk('обновлённая_модель')
```
Можно многократно использовать цикл обучения для пошагового дообучения модели на разных данных и для добавления новых сущностей.

* [Какой тип информации содержится в токене?](https://habr.com/ru/articles/531940/)

`is_alpha` - содержит ли **только** буквенные символы </br>
`is_punct` - является ли токен знаком пунктуации </br>
`like_num` - является ли токен числом

```python
token.is_alpha for token in doc
```

* Вывести все токены, предшествующие точке

```python
for token in doc:
    if token.i+1 < len(doc):
        next_token = doc[token.i+1]
        if next_token == '.'
        print(token.text)
```
* Построение синтаксического дерева

В spaCy получить получить дерево зависимостей для предложения можно с помощью метода `.print_tree()`, а также:

```python
from spacy import displacy
displacy.render(doc, style='dep', jupyter=True)
```
* Расшифровка названия тегов `spacy.explain('aux')`

    * aux = auxiliary = частица
    * propn = proper noun = имя собственное
    * root = корневой токен или главное слово в предложение

* Выделение именованных сущностей

Для получения списка именованных сущностей используется `doc.ents`, а для получения метки этой сущности `token.label_` </br>
Функция **displacy** позволяет наглядно обозначить сущности прямо в тексте:

```python
from spacy import displacy
displacy.render(doc, style='ent', jupyter=True)
```
* Создание собственного шаблона для поиска текста

Модуль spaCy содержит инструмент, который позволяет строить свои собственные шаблоны для поиска текста. Можно искать слова определённой части речи, все формы слова по его начальной форме, делать проверку на тип содержимого в токене.

PARAM | Описание
--- | ---
TEXT | Поиск токенов по полному сооетветствию с указанным текстом, внутри параметра можно использовать регулярные выражения
LOWER | Позволяет игнорировать регистр + аналогично TEXT
LIKE_NUM, LIKE_URL, LIKE_EMAIL | Поиск чисел, ссылок или электронных адресов
POS | Поиск токенов определённой части речи
LEMMA | Поиск всех форм слова от указанной начальной формы
OP | Особый ключ, указывающий на количество употреблений токена

```python
from spacy.matcher import Matcher

nlp = spacy.load('ru_core_load_md')
matcher = Matcher(nlp.vocab)

pattern = [
    {'IS_DIGIT': True },
    ...
]

matcher.add('new_pattern', None, pattern)
doc = nlp(comments)
mathces = matcher(doc)

for match_id, start, end in matches:
    matched_span = doc[start:end]
    print(matched_span)

```


* [Ещё статься на Хабр](https://habr.com/ru/companies/otus/articles/755584/)
* [Статься про spacy на Proglib](https://proglib.io/p/lyublyu-i-nenavizhu-analiz-emocionalnoy-okraski-teksta-s-pomoshchyu-python-2020-11-13)

```python
import spacy
from spacy.training.example import Example
import pandas as pd

nlp = spacy.load('ru_core_news_md')

new_entities = [
    ('40 млн руб под 9.8%', 'MONEY'),
    ('овердрафт', 'PROD'),
    ('реф', 'PROD'),
    ('лизинг', 'PROD'),
]

ner = nlp.get_pipe('ner')
ner.add_label(label)

excel_file_path = 'путь_к_файлу'
df = pd.read_excel(excel_file_path)

def prepare_data(comment, entities_str):
    entities = [tuple(entity.split(',')) for entity in entity_str.split(',')]
    doc = nlp.make_doc(comment)
    gold_dict = {'entities': [(int(start), int(stop), label) for start, stop, label in entities]}
    example = Example.from_dict(doc, gold_dict)
    return example

for index, row in df.iterrows():
    comment = row['Комментарий']
    entities_str = row['Entities']
    example = prepare_data(comment, entities_str)
    nlp.update([example], drop=0.5, losses={})

nlp.to_disk('обновлённая_модель')
```
Переменная **gold** в контексте обучения моделей spaCy используется для представления золотых стандартов, то есть корректных аннотаций или разметки, которую вы хотели бы, чтобы модель воспроизвела или приблизила при обучении. Это эталонная разметка, с которой вы хотите сравнивать результаты обучения. </br>
Объекст **gold** содержит информацию о сущностях для данного комментария. Он передаётся в **Example.from_dict**, чтобы создать пример для обучения модели **nlp.update.**</br>
**from_dict** - это метод класса **Example**, который используется для создания экземпляра класса **Example** из словаря. </br>
Для оценки точности на тестовых данных можно использовать встроенные функции spaCy, такие как `spacy.evaluate`
```python
import spacy

nlp = spacy.load('path_to_trained_model')

test_data = [
    ('Текст комментария', {'entities': [(start, end, 'label'), ... ]}),
    ...
]

results = spacy.evaluate(nlp, test_data)
print(results)
```
Здесь **test_data** содержит текст комментариев и их золотые стандарты (аннотации), а функция **spacy.evaluate** возвращает словарь с результатми оценки, включая точность, полноту, F-меру и другие метрики.



In [ ]:
import pandas as pd

results_df = pd.DataFrame(columns = ['text', 'dep_', 'head'])

text = 'бк с ГД, имеется овердрафт в ВТБ 40 млн руб под 9,8%, реф и вкл не интересует, полностью устраивает овердрафт. По иным продуктам потребности нет, кроме лизинга. В лизинг интересует оборудование, пока конкретную информацию дать не может, тк еще не выбрали, что нужно, сейчас руководство ездит по выставкам и выбирает, ближайшая поездка в Турцию, сумма лизинга от 10 млн руб, как выберут, лично'
doc = nlp(text)

for token in doc:
    results_df.loc[len(results_df)] = [token.lemma_.lower(), token.dep_, token.head.text.lower()]

results_df[results_df.text.isin(['лизинг', 'овердрафт'])] #токен-глава

,text,dep_,head
5,овердрафт,nsubj,имеется
23,овердрафт,nsubj,устраивает
32,лизинг,obl,нет
35,лизинг,obl,интересует
68,лизинг,nmod,сумма


### Dependency Parser

* DependencyParser.create_optimizer
```python
parser = nlp.add_pipe('parser')
optimizer = parser.create_optimizer()
```
* DependencyParser.update
```python
parser = nlp.add_pipe('parser')
optimizer = nlp.initialize()
losses = parser.update(examples, sgd=optimizer)
```
* EntityRecognizer.create_optimizer
```pyton
ner = nlp.add_pipe('ner')
optimizer = ner.create_optimizer()
```
* EntityRecognizer.update
```python
ner = nlp.add_pipe('ner')
optimizer = nlp.initialize()
losses = ner.update(examples, sgd=optimizer)
```
* EntityRecognizer.labels
```python
ner.add_label('MY_LABEL')
assert 'MY_LABEL' in ner.labels
```

В списке `heads` в аннотации зависимостей каждому токену в предложении сопоставляется индекс другого токена, который является его родителем или головным элементом.

```python
train_data = [
    ('Мы редко ходим в кино', {'heads': [2, 2, 2, 0, 2], 'deps': ['nsubj', 'advmod', 'ROOT', 'prep', 'advmod']}),
    ('Депозиты не интересуют', {'heads': [3, 3, 0], 'deps': ['nsubj', 'neg', 'ROOT']})

]
```

```python
for token in doc:
    in not token.ent_type_:
```

```python
from spacy.pipeline import EntityRecognizer

disabled_entities = ['ORG', 'LOC']

for ent in disabled_entities:
    nlp.entity.remove_label(ent)

available_entities = nlp.get_pipe('ner').labels
print('Доступные сущности:', available_entities)
```

